In [1]:
import gc
import pickle
import numpy as np
import torch

def datato3d(arrays):#list of np arrays, NULL*3*100
    output=list()
    for i in arrays:
        i=np.transpose(i,(0,2,1))
        output.append(i)
    return output


def udflip(X_nparray, y_nparray, shuffle=True):

    if X_nparray.shape[2] == 4:
        if np.std(X_nparray[:, 0, :]) > np.std(X_nparray[:, -1, :]):
            print("Detected special info in first column, swapping...")
            X_nparray = np.concatenate((X_nparray[:, 1:, :], X_nparray[:, 0:1, :]), axis=1)
    
    X_flipped = np.flip(X_nparray, axis=2)  

    X_aug = np.vstack((X_nparray, X_flipped))
    y_aug = np.hstack((y_nparray, y_nparray))  

    if shuffle:
        shuffle_idx = np.random.permutation(X_aug.shape[0])
        return X_aug[shuffle_idx], y_aug[shuffle_idx]
    else:
        return X_aug, y_aug

with open('../data_61_26_ROIV2.pkl','rb') as f:
    data=pickle.load(f)


dataList=datato3d([data['X_train'],data['X_test']])



In [2]:
data['X_train'].shape, data['X_test'].shape,data['X_train'][4].shape

((3496641, 100, 4), (1497440, 100, 4), (100, 4))

In [3]:
data['X_train'][4]

array([[-50.86827  ,  -5.1718316, -11.912324 ,   0.       ],
       [-50.886658 ,  -5.4194384, -11.963684 ,   0.       ],
       [-50.911953 ,  -5.687157 , -12.018323 ,   0.       ],
       [-50.926834 ,  -5.954355 , -12.07456  ,   0.       ],
       [-50.913975 ,  -6.200402 , -12.130718 ,  20.       ],
       [-50.85936  ,  -6.430222 , -12.192032 ,  20.       ],
       [-50.769756 ,  -6.6747646, -12.26447  ,  20.       ],
       [-50.662407 ,  -6.9200387, -12.3379   ,  20.       ],
       [-50.554653 ,  -7.151108 , -12.401925 ,  20.       ],
       [-50.4517   ,  -7.367921 , -12.4543   ,  20.       ],
       [-50.331234 ,  -7.6009817, -12.509045 ,  20.       ],
       [-50.201836 ,  -7.8392177, -12.557598 ,  20.       ],
       [-50.074444 ,  -8.068485 , -12.589779 ,  20.       ],
       [-49.95623  ,  -8.280725 , -12.597824 ,  20.       ],
       [-49.827026 ,  -8.510864 , -12.589802 ,  20.       ],
       [-49.689793 ,  -8.750706 , -12.565458 ,  20.       ],
       [-49.55394  ,  -8

In [4]:
X_train=dataList[0]
X_test=dataList[1]
# X_train, y_train = X_train[:, :, ::5], data['y_train'][:]
# X_test, y_test = X_test[:, :, ::5], data['y_test'][:]
X_train, y_train = X_train[:, :, ::1], data['y_train'][:]
X_test, y_test = X_test[:, :, ::1], data['y_test'][:]

y_test_list=data['y_test'].tolist()

NCLASS=max(y_test_list)+1

X_train,y_train=udflip(X_train,y_train,shuffle=True)
X_test,y_test=udflip(X_test,y_test,shuffle=False)

X_train=torch.from_numpy(X_train)#data['X_train'])
y_train=torch.from_numpy(y_train.astype(np.int32))#data['y_train'])

X_test=torch.from_numpy(X_test)
y_test=torch.from_numpy(y_test.astype(np.int32))  


In [5]:

del data,dataList
gc.collect()
print('data loaded!')
print('X_train_shape',X_train.size())
print('X_test_shape',X_test.size())
print('classes: ', NCLASS)

data loaded!
X_train_shape torch.Size([6993282, 4, 100])
X_test_shape torch.Size([2994880, 4, 100])
classes:  15


In [6]:
X_train.shape, X_train[10:20,:,:].shape, y_train.shape, y_train[10:20].shape

(torch.Size([6993282, 4, 100]),
 torch.Size([10, 4, 100]),
 torch.Size([6993282]),
 torch.Size([10]))

In [7]:
X_train[14], y_train[14]

(tensor([[-22.7798, -23.2095, -23.6488, -24.1066, -24.5564, -25.0120, -25.4260,
          -25.8552, -26.2227, -26.5948, -26.9435, -27.3249, -27.6934, -27.9871,
          -28.2688, -28.5298, -28.8139, -29.0853, -29.3537, -29.6324, -29.8968,
          -30.1894, -30.4526, -30.7249, -30.9713, -31.1477, -31.3044, -31.4400,
          -31.5794, -31.6990, -31.7768, -31.8354, -31.8841, -31.9207, -31.9668,
          -32.0530, -32.1843, -32.3450, -32.5947, -32.8801, -33.1978, -33.5565,
          -33.9500, -34.3913, -34.8791, -35.3947, -35.9442, -36.4851, -37.0730,
          -37.6075, -38.1614, -38.6716, -39.2143, -39.7491, -40.3289, -40.8770,
          -41.4341, -41.9638, -42.4828, -43.0007, -43.5003, -44.0078, -44.5013,
          -45.0275, -45.5446, -46.1130, -46.6539, -47.2165, -47.7126, -48.1638,
          -48.5804, -49.0115, -49.4411, -49.9411, -50.4186, -50.9084, -51.3395,
          -51.7195, -52.0617, -52.3882, -52.7167, -53.1041, -53.5042, -53.9351,
          -54.3428, -54.6618, -54.9393, 

In [8]:
import torch

def compute_fiber_roi(fiber_data):
    """
    Compute the unique ROI classification for each fiber, removing ROI values of 0.

    Parameters:
        fiber_data: Tensor of shape (b, 4, 100), where the last dimension represents ROI classification.

    Returns:
        roi_list: List[Tensor], each element contains a fiber's unique ROI classifications (deduplicated, excluding 0).
    """
    # Extract ROI classification data (b, 100)
    roi_data = fiber_data[:, 3, :]

    # Remove 0 and get unique ROI classifications
    roi_list = [torch.unique(roi[roi != 0]) for roi in roi_data]

    return roi_list

# Example data
b, num_points = 5, 100  # 5 fibers, each with 100 points
fiber_data = X_train[:20, :, :]  # Extract ROI classifications
labels = y_train[:20]

# Compute ROI classifications for each fiber
fiber_rois = compute_fiber_roi(fiber_data)

# Print results
for i, rois in enumerate(fiber_rois):
    print(f"Fiber {i + 1}: {rois.tolist()} cluster: {labels[i]}")


Fiber 1: [487.0, 692.0] cluster: 11
Fiber 2: [47.0] cluster: 4
Fiber 3: [422.0, 648.0] cluster: 12
Fiber 4: [151.0, 191.0, 691.0] cluster: 1
Fiber 5: [649.0] cluster: 13
Fiber 6: [] cluster: 4
Fiber 7: [27.0] cluster: 4
Fiber 8: [228.0] cluster: 6
Fiber 9: [159.0, 196.0] cluster: 6
Fiber 10: [36.0, 37.0, 48.0, 49.0] cluster: 4
Fiber 11: [156.0, 691.0] cluster: 2
Fiber 12: [146.0, 152.0] cluster: 14
Fiber 13: [221.0, 228.0] cluster: 6
Fiber 14: [] cluster: 6
Fiber 15: [96.0, 145.0, 691.0] cluster: 0
Fiber 16: [111.0] cluster: 14
Fiber 17: [151.0, 152.0, 153.0, 691.0] cluster: 1
Fiber 18: [30.0, 36.0, 37.0] cluster: 4
Fiber 19: [] cluster: 3
Fiber 20: [647.0] cluster: 12


In [9]:
import torch

def compute_cluster_roi(X_train, y_train, num_of_class, threshold=0.00000000002):
    """
    Compute ROI classification for each cluster based on the rule that 40% of fibers pass through an ROI (ignoring 0).
    Also, print the ROI each fiber passes through along with its corresponding cluster.

    Parameters:
        X_train: Tensor, shape (b, 4, 100), where the 4th dimension represents ROI classification.
        y_train: Tensor, shape (b,), containing each fiber's cluster label.
        num_of_class: int, total number of clusters.
        threshold: float, threshold setting (default is 0.4, i.e., 40%).

    Returns:
        cluster_rois: List[Tensor], each element contains the ROI classification of the cluster (deduplicated & filtered by threshold).
    """
    # Extract fiber ROI classification information (b, 100)
    roi_data = X_train[:, 3, :]

    # Compute unique ROI classifications for each fiber (remove duplicates & ignore 0)
    fiber_rois = [torch.unique(roi[roi != 0]) for roi in roi_data]
    # Compute ROI for each cluster
    cluster_rois = []
    for cluster_id in range(num_of_class):
        # Get fibers belonging to the current cluster
        cluster_fibers = [fiber_rois[i] for i in range(len(y_train)) if y_train[i] == cluster_id]

        if not cluster_fibers:  # If no fibers belong to this cluster, skip
            cluster_rois.append(torch.tensor([]))
            continue

        # Aggregate all ROI classifications from fibers
        all_rois = torch.cat(cluster_fibers)  # Concatenate all fiber ROIs
        unique_rois, counts = torch.unique(all_rois, return_counts=True)  # Count occurrences of each ROI

        # Compute the occurrence ratio
        fiber_count = len(cluster_fibers)  # Total number of fibers in this cluster
        roi_ratio = counts.float() / fiber_count  # Compute the proportion of fibers passing through each ROI

        # Select ROIs that meet the 40% threshold
        selected_rois = unique_rois[roi_ratio >= threshold]
        cluster_rois.append(selected_rois)

    return cluster_rois

# Example data
num_of_class = NCLASS  # Total number of clusters
num_samples = 1000000
x_train_sub = X_train[:num_samples, :, :]  # Select the first 100 fibers
y_train_sub = y_train[:num_samples]  # Select the first 100 fiber cluster labels

# Compute cluster-level ROI classification
cluster_roi_classes = compute_cluster_roi(x_train_sub, y_train_sub, num_of_class)

# Print cluster-level ROI classification results
print("\n===== Cluster-Level ROI Classification =====")
for cluster_id, rois in enumerate(cluster_roi_classes):
    print(f"Cluster {cluster_id}: {rois.tolist()}")



===== Cluster-Level ROI Classification =====
Cluster 0: [21.0, 22.0, 23.0, 24.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 100.0, 101.0, 116.0, 141.0, 142.0, 143.0, 144.0, 145.0, 146.0, 147.0, 148.0, 149.0, 150.0, 156.0, 157.0, 158.0, 159.0, 160.0, 161.0, 214.0, 215.0, 217.0, 218.0, 219.0, 220.0, 221.0, 335.0, 338.0, 339.0, 342.0, 343.0, 691.0]
Cluster 1: [23.0, 24.0, 96.0, 110.0, 111.0, 112.0, 113.0, 114.0, 115.0, 116.0, 117.0, 118.0, 119.0, 120.0, 121.0, 122.0, 134.0, 135.0, 136.0, 137.0, 138.0, 139.0, 140.0, 145.0, 146.0, 150.0, 151.0, 152.0, 153.0, 154.0, 155.0, 161.0, 182.0, 183.0, 185.0, 186.0, 187.0, 188.0, 189.0, 190.0, 191.0, 192.0, 193.0, 194.0, 216.0, 219.0, 315.0, 316.0, 317.0, 318.0, 319.0, 320.0, 332.0, 335.0, 336.0, 338.0, 339.0, 342.0, 343.0, 344.0, 691.0]
Cluster 2: [23.0, 24.0, 92.0, 93.0, 94.0, 96.0, 115.0, 116.0, 120.0, 138.0, 139.0, 140.0, 141.0, 142.0, 143.0, 145.0, 146.0, 153.0, 154.0, 155.0, 156.0, 157.0, 158.0, 161.0, 190.0, 194.0, 214.0, 215.0, 217.0, 218.0, 219.0,

In [11]:
# Print cluster-level ROI classification results
print("\n===== Cluster-Level ROI Classification =====")
for cluster_id, rois in enumerate(cluster_roi_classes):
    print(f"Cluster {cluster_id}: {rois.tolist()}, size {len(rois.tolist())}")


===== Cluster-Level ROI Classification =====
Cluster 0: [21.0, 22.0, 23.0, 24.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 100.0, 101.0, 116.0, 141.0, 142.0, 143.0, 144.0, 145.0, 146.0, 147.0, 148.0, 149.0, 150.0, 156.0, 157.0, 158.0, 159.0, 160.0, 161.0, 214.0, 215.0, 217.0, 218.0, 219.0, 220.0, 221.0, 335.0, 338.0, 339.0, 342.0, 343.0, 691.0], size 42
Cluster 1: [23.0, 24.0, 96.0, 110.0, 111.0, 112.0, 113.0, 114.0, 115.0, 116.0, 117.0, 118.0, 119.0, 120.0, 121.0, 122.0, 134.0, 135.0, 136.0, 137.0, 138.0, 139.0, 140.0, 145.0, 146.0, 150.0, 151.0, 152.0, 153.0, 154.0, 155.0, 161.0, 182.0, 183.0, 185.0, 186.0, 187.0, 188.0, 189.0, 190.0, 191.0, 192.0, 193.0, 194.0, 216.0, 219.0, 315.0, 316.0, 317.0, 318.0, 319.0, 320.0, 332.0, 335.0, 336.0, 338.0, 339.0, 342.0, 343.0, 344.0, 691.0], size 61
Cluster 2: [23.0, 24.0, 92.0, 93.0, 94.0, 96.0, 115.0, 116.0, 120.0, 138.0, 139.0, 140.0, 141.0, 142.0, 143.0, 145.0, 146.0, 153.0, 154.0, 155.0, 156.0, 157.0, 158.0, 161.0, 190.0, 194.0, 214.0, 215.0, 21

In [10]:
# import torch

# def compute_cluster_roi(X_train, y_train, num_of_class, threshold=0.4):
#     """
#     Compute ROI class information at the cluster level and ignore fibers without classification.
#     Also, count the number of valid fibers within each cluster.

#     Parameters:
#         X_train: Tensor, shape (b, 4, 100), where the 4th column represents ROI.
#         y_train: Tensor, shape (b,), containing each fiber's cluster label.
#         num_of_class: int, number of clusters.
#         threshold: float, default threshold is 40%.

#     Returns:
#         cluster_rois: List[Tensor], cluster ROI information.
#         cluster_fiber_counts: List[int], where each element represents the number of valid fibers within the cluster.
#     """
#     # Extract fiber ROI classification information (b, 100)
#     roi_data = X_train[:, 3, :]

#     # Compute unique ROI classifications for each fiber (remove duplicates & ignore 0)
#     fiber_rois = [torch.unique(roi[roi != 0]) for roi in roi_data]

#     # Filter out fibers without any classification (i.e., fibers where fiber_rois is empty)
#     valid_indices = [i for i in range(len(fiber_rois)) if fiber_rois[i].numel() > 0]
#     valid_fiber_rois = [fiber_rois[i] for i in valid_indices]
#     valid_y_train = y_train[valid_indices]  # Filtered y_train

#     # Print ROI and cluster information for each valid fiber
#     # print("\n===== Fiber ROI & Cluster Information (Ignoring Unclassified Fibers) =====")
#     # for i, idx in enumerate(valid_indices):
#     #     print(f"Fiber {idx + 1} (Cluster {valid_y_train[i].item()}): {valid_fiber_rois[i].tolist()}")

#     # Compute ROI and fiber count for each cluster
#     cluster_rois = []
#     cluster_fiber_counts = []  # Store the number of valid fibers in each cluster

#     for cluster_id in range(num_of_class):
#         # Get fibers belonging to the current cluster
#         cluster_fibers = [valid_fiber_rois[i] for i in range(len(valid_y_train)) if valid_y_train[i] == cluster_id]

#         cluster_fiber_counts.append(len(cluster_fibers))  # Record the number of valid fibers in the cluster

#         if not cluster_fibers:  # If no fibers belong to this cluster, skip
#             cluster_rois.append(torch.tensor([]))
#             continue

#         # Aggregate all ROI classifications from fibers
#         all_rois = torch.cat(cluster_fibers)  # Concatenate all fiber ROIs
#         unique_rois, counts = torch.unique(all_rois, return_counts=True)  # Count occurrences of each ROI

#         # Compute the occurrence ratio
#         fiber_count = len(cluster_fibers)  # Total number of fibers in this cluster
#         roi_ratio = counts.float() / fiber_count  # Compute the proportion of fibers passing through each ROI

#         # Select ROIs that meet the 40% threshold
#         selected_rois = unique_rois[roi_ratio >= threshold]
#         cluster_rois.append(selected_rois)

#     return cluster_rois, cluster_fiber_counts

# # Example data
# num_of_class = NCLASS  # Total number of clusters
# num_samples = 10000
# x_train_sub = X_train[:num_samples, :, :]  # Select the first 1000 fibers
# y_train_sub = y_train[:num_samples]  # Select the first 1000 fiber cluster labels

# # Compute cluster-level ROI classification and fiber count
# cluster_roi_classes, cluster_fiber_counts = compute_cluster_roi(x_train_sub, y_train_sub, num_of_class, threshold=0.1)

# # Print cluster-level ROI classification results and fiber counts
# print("\n===== Cluster-Level ROI Classification and Fiber Count =====")
# for cluster_id, (rois, fiber_count) in enumerate(zip(cluster_roi_classes, cluster_fiber_counts)):
#     print(f"Cluster {cluster_id} (Valid Fiber Count: {fiber_count}): {rois.tolist()}")


In [90]:
import torch

def compute_fiber_roi(fiber_data):
    """
    Compute the unique ROI category numbers for each fiber, removing points where ROI = 0.

    Parameters:
        fiber_data: Tensor of shape (b, 4, 100), where the last dimension represents ROI category labels.

    Returns:
        roi_list: List[Tensor], each element is a tensor containing the unique ROI category numbers for a fiber 
                  (duplicates removed, and 0 values excluded).
    """
    roi_data = fiber_data[:, 3, :]  # Extract ROI dimension (b, 100)
    roi_list = [torch.unique(roi[roi != 0]) for roi in roi_data]  # Compute unique ROIs
    return roi_list

import torch

import torch

import torch

import torch

def dice_loss_fiber(fiber_data, fiber_pred, cluster_roi_true, smooth=1e-6):
    """
    Compute the fiber-level ROI Dice Loss based only on the predicted cluster.

    Parameters:
        fiber_data: Tensor, shape (b, 4, 100), containing fiber ROI information.
        fiber_pred: Tensor, shape (b,), model-predicted fiber cluster labels.
        cluster_roi_true: List[Tensor], ground-truth ROI categories for each cluster (each cluster has one tensor).
        smooth: float, a small value to avoid division by zero.

    Returns:
        dice_losses_per_batch: Tensor, shape (b,), containing Dice Loss for each batch element.
    """
    fiber_rois = compute_fiber_roi(fiber_data)  # Compute ROI categories for each fiber
    batch_size = fiber_data.shape[0]  # Get batch size
    fiber_losses_per_batch = []

    for batch_idx in range(batch_size):  # Iterate over each batch
        ROI_fiber = fiber_rois[batch_idx]  # Fiber-level ROI
        pred_cluster = fiber_pred[batch_idx].item()  # Model-predicted cluster
        
        # Get predicted cluster-level ROI
        ROI_cluster_pred = cluster_roi_true[pred_cluster] if pred_cluster < len(cluster_roi_true) else torch.tensor([])

        if ROI_fiber.numel() == 0 and ROI_cluster_pred.numel() == 0:
            fiber_losses_per_batch.append(torch.tensor(0.0))  # If both are empty, loss = 0
        elif ROI_fiber.numel() == 0 or ROI_cluster_pred.numel() == 0:
            fiber_losses_per_batch.append(torch.tensor(1.0))  # If only one is empty, loss = 1
        else:
            # Compute Dice Loss based on fiber ROI and predicted cluster ROI
            intersection = (ROI_fiber.unsqueeze(1) == ROI_cluster_pred.unsqueeze(0)).sum().float()
            dice_score = (2. * intersection + smooth) / (ROI_fiber.numel() + ROI_cluster_pred.numel() + smooth)
            fiber_losses_per_batch.append(1 - dice_score)  # 1 - Dice coefficient = Dice Loss

    return torch.tensor(fiber_losses_per_batch)  # Shape: (b,)





# Example data
b = 2  # Number of fibers
num_of_classes = 14  # Number of clusters
num_points = 100  # Each fiber has 100 points

# Generate fiber data (b, 4, 100) (last dimension represents ROI category)
fiber_data = X_train[:10, :, :]  # ROI values range from 0 to 693
# Generate predicted cluster labels for fibers
fiber_pred = y_train[:10]
print(fiber_pred)
fiber_pred[0] = 11
# Generate ground-truth ROI categories for each cluster (each cluster has one tensor)
cluster_roi_true = cluster_roi_classes

# Compute Dice Loss
loss = dice_loss_fiber(fiber_data, fiber_pred, cluster_roi_true)
fiber_rois_profile = compute_fiber_roi(fiber_data)
print(f"Fiber-Level Dice Loss: {loss}")
print(cluster_roi_true[11])
print(fiber_rois_profile[0])
print(fiber_pred[0])
# 9: 0.9167

tensor([11,  4, 12,  1, 13,  4,  4,  6,  6,  4], dtype=torch.int32)
Fiber-Level Dice Loss: tensor([0.9167, 0.9930, 0.9708, 0.9062, 0.9836, 1.0000, 0.9930, 0.9930, 0.9861,
        0.9723])
tensor([367., 368., 438., 439., 441., 459., 460., 461., 480., 481., 483., 484.,
        485., 486., 487., 488., 490., 491., 496., 497., 498., 499., 500., 501.,
        502., 503., 506., 531., 534., 535., 536., 537., 539., 562., 563., 564.,
        677., 680., 681., 683., 684., 685., 687., 688., 689., 692.])
tensor([487., 692.])
tensor(11, dtype=torch.int32)


In [88]:
import torch

def compute_fiber_roi(fiber_data):
    """
    Compute the unique ROI category numbers for each fiber, removing points where ROI = 0.

    Parameters:
        fiber_data: Tensor of shape (b, 4, 100), where the last dimension represents ROI category labels.

    Returns:
        roi_list: List[Tensor], each element is a tensor containing the unique ROI category numbers for a fiber 
                  (duplicates removed, and 0 values excluded).
    """
    roi_data = fiber_data[:, 3, :]  # Extract ROI dimension (b, 100)
    roi_list = [torch.unique(roi[roi != 0]) for roi in roi_data]  # Compute unique ROIs
    return roi_list

import torch

import torch

import torch

import torch

import torch

import torch

def dice_loss_fiber(fiber_data, fiber_pred, fiber_gt, cluster_roi_true, smooth=1e-6):
    """
    Compute the fiber-level ROI Dice Loss, considering fiber prediction correctness.

    Parameters:
        fiber_data: Tensor, shape (b, 4, 100), containing fiber ROI information.
        fiber_pred: Tensor, shape (b,), model-predicted fiber cluster labels.
        fiber_gt: Tensor, shape (b,), ground-truth fiber cluster labels.
        cluster_roi_true: List[Tensor], ground-truth ROI categories for each cluster (each cluster has one tensor).
        smooth: float, a small value to avoid division by zero.

    Returns:
        dice_losses_per_batch: Tensor, shape (b,), containing Dice Loss for each batch element.
    """
    fiber_rois = compute_fiber_roi(fiber_data)  # Compute ROI categories for each fiber
    batch_size = fiber_data.shape[0]  # Get batch size
    fiber_losses_per_batch = []

    for batch_idx in range(batch_size):  # Iterate over each batch
        ROI_fiber = fiber_rois[batch_idx]  # Fiber-level ROI
        pred_cluster = fiber_pred[batch_idx].item()  # Predicted fiber cluster
        gt_cluster = fiber_gt[batch_idx].item()  # Ground-truth fiber cluster
        
        ROI_cluster_pred = cluster_roi_true[pred_cluster] if pred_cluster < len(cluster_roi_true) else torch.tensor([])
        ROI_cluster_gt = cluster_roi_true[gt_cluster] if gt_cluster < len(cluster_roi_true) else torch.tensor([])

        if ROI_fiber.numel() == 0 and ROI_cluster_pred.numel() == 0:
            fiber_losses_per_batch.append(torch.tensor(0.0))  # If both are empty, loss = 0
        elif ROI_fiber.numel() == 0 or ROI_cluster_pred.numel() == 0:
            fiber_losses_per_batch.append(torch.tensor(1.0))  # If only one is empty, loss = 1
        else:
            # 计算 ROI_fiber 与 cluster 预测的 ROI 之间的 Dice Loss
            intersection = (ROI_fiber.unsqueeze(1) == ROI_cluster_pred.unsqueeze(0)).sum().float()
            dice_score = (2. * intersection + smooth) / (ROI_fiber.numel() + ROI_cluster_pred.numel() + smooth)
            dice_loss = 1 - dice_score  # 1 - Dice coefficient = Dice Loss

            # 如果 fiber 分类错误，增加 loss 惩罚
            if pred_cluster != gt_cluster:
                dice_loss += 0.5  # 额外增加损失，惩罚错误分类
            
            fiber_losses_per_batch.append(dice_loss)

    return torch.tensor(fiber_losses_per_batch)  # Shape: (b,)







# Example data
b = 2  # Number of fibers
num_of_classes = 14  # Number of clusters
num_points = 100  # Each fiber has 100 points

# Generate fiber data (b, 4, 100) (last dimension represents ROI category)
fiber_data = X_train[10:20, :, :]  # ROI values range from 0 to 693
# Generate predicted cluster labels for fibers
fiber_pred_gt = y_train[10:20]
fiber_pred = torch.tensor(fiber_pred_gt)
test_index = 1
fiber_pred[test_index] = 14
# Generate ground-truth ROI categories for each cluster (each cluster has one tensor)
cluster_roi_true = cluster_roi_classes

# Compute Dice Loss
loss = dice_loss_fiber(fiber_data, fiber_pred, fiber_pred_gt, cluster_roi_true)
fiber_rois_profile = compute_fiber_roi(fiber_data)
print(f"Fiber-Level Dice Loss: {loss}")
print(cluster_roi_true[test_index])
print(fiber_pred_gt)
print(fiber_rois_profile[test_index])
print(fiber_pred[test_index])
print(fiber_pred_gt[test_index])
# 9: 0.9167

Fiber-Level Dice Loss: tensor([0.9000, 0.9942, 0.9861, 1.0000, 0.8667, 0.9971, 0.8769, 0.9792, 1.0000,
        0.9853])
tensor([ 23.,  24.,  96., 110., 111., 112., 113., 114., 115., 116., 117., 118.,
        119., 120., 121., 122., 134., 135., 136., 137., 138., 139., 140., 145.,
        146., 150., 151., 152., 153., 154., 155., 161., 182., 183., 185., 186.,
        187., 188., 189., 190., 191., 192., 193., 194., 216., 219., 315., 316.,
        317., 318., 319., 320., 332., 335., 336., 338., 339., 342., 343., 344.,
        691.])
tensor([ 2, 14,  6,  6,  0, 14,  1,  4,  3, 12], dtype=torch.int32)
tensor([146., 152.])
tensor(14, dtype=torch.int32)
tensor(14, dtype=torch.int32)


C:\Users\bohan\AppData\Local\Temp\ipykernel_24068\427431408.py:89: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  fiber_pred = torch.tensor(fiber_pred_gt)


In [17]:
import torch.utils.data as utils

kwargs = {}
trn_set=utils.TensorDataset(X_train,y_train)
trn_loader=utils.DataLoader(trn_set,batch_size=2,shuffle=True,**kwargs)

# tst_set=utils.TensorDataset(X_test,y_test)
# tst_loader=utils.DataLoader(tst_set,batch_size=2,shuffle=False,**kwargs)

In [18]:
def compute_tract_anatomical_profile(fiber_roi):
    """
    """
    unique, counts = np.unique(fiber_roi, return_counts=True)
    roi_profile = dict(zip(unique, counts / len(fiber_roi)))  # 计算占比
    return roi_profile


In [19]:
for batch_idx,(data,target) in enumerate(trn_loader):
    # data = data[0]
    # target = target[0]
    print(data.shape)
    print(data.shape)
    print(data[:, 3:4, :].shape)
    print(data[:, 3:4, :])
    roi_profile = compute_tract_anatomical_profile(data[:, 3:4, :])
    # print(roi_profile)
    # print(target.shape)
    # print(data)
    # print(target)
    break

torch.Size([2, 4, 100])
torch.Size([2, 4, 100])
torch.Size([2, 1, 100])
tensor([[[158., 158., 158.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0., 157., 157., 157., 157.,   0.,   0.,   0.,   0.,   0., 157.,
          157.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.]],

        [[505., 505., 505., 505., 505., 505., 505., 505., 505., 505., 505.,
          505., 505., 505., 505., 505., 504., 504., 504., 504., 504., 504.,
          504.,   0., 504., 504., 504.,   0.,   0.,   0.,   0.,   0.,   0